给定一个需求说明书，从中选择一个可以产生创新点的需求，将它重新描述为专利发明点的问题。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
template_requirements = '''你是个非常优秀的程序员，你的任务是理解软件产品的需求说明书，并从中选择一个可以作为专利发明点的功能。
这个产品的核心功能是流程挖掘和任务挖掘，分别从业务日志和UI操作日志中还原出业务流程并进行分析。
创新的功能不太常见的功能，这样容易找到创新的解决方案。也可以是用于解决常见问题，但是在新的业务场景下，或者选择新的算法，能够实现创新的解决方案。

需求说明: {requirements}

请一步步思考，确保正确理解了所有的功能需求，然后选择一个最有创新潜力的功能点。
重新描述这个功能点，包含上下文相关信息，以及为什么它具备创新潜力：'''

prompt_requirements = PromptTemplate(input_variables=['requirements'], template=template_requirements)
requirements_chain = LLMChain(llm=llm, prompt=prompt_requirements, output_key='feature')

In [4]:
template_feature = '''你们是三个著名的产品经理，你们的任务是根据完整的需求描述提供一个创新的解决思路。
请你们每个人提供一个，详细说明这是一个什么样的问题，为什么要解决它，可以采用什么样的思路来解决，以及如何体现创新性。

功能说明: {feature}

请你们各自生成一个思路描述'''

prompt_feature = PromptTemplate(input_variables=['feature'], template=template_feature)
feature_chain = LLMChain(llm=llm, prompt=prompt_feature, output_key='solution')

In [5]:
template_selection = '''你是一个著名的程序员，请根据软件功能描述和多个解决思路，选择一个最具创新性的解决思路。
该解决思路要能够通过代码实现出来，并具有一定的创新性，以便申请专利。

程序说明书: {feature}
可选的解决方案: {solution}

请一步步思考，重新描述最好的解决方案，要包含问题的描述，解决它的价值，以及解决它的创新思路：'''

prompt_selection = PromptTemplate(input_variables=['feature', 'solution'], template=template_selection)
selection_chain = LLMChain(llm=llm, prompt=prompt_selection, output_key='selection')

In [6]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[requirements_chain, feature_chain, selection_chain],
    input_variables=["requirements"],
    # Here we return multiple variables
    output_variables=["feature", "solution", "selection"],
    verbose=True)

1、申请试用（已完成）



2、开通租户

（1）在CRM中填写“申请授权许可”表单，需求详情：http://192.168.0.201:8088/zentao/story-view-6675.html

（2）许可申请提交后，调取工作台接口，工作台将许可申请信息传给RPM服务，由RPM服务开通租户（注：租户管理员账号不占用“分析用户数”许可数量）

          - 未开通租户的申请内容包括：艺赛旗ID、许可数量、许可到期时间

          - 已开通过的租户申请内容包括：租户编号、许可数量、许可到期时间

（3）工作台定时任务轮询每个艺赛旗ID在RPM服务端的租户信息

          - 若租户开通成功，RPM服务提供的租户信息内容包括：租户编号、租户别名、租户域名、租户到期时间、许可数量

          - 若租户开通失败，RPM服务需提供失败原因

（4）葛亮定时任务向工作台查询申请结果，回传至crm中

（5）租户开通成功后，工作台需发送短信或邮件给相应的艺赛旗ID通知其租户开通成功

         - 短信内容如下：【艺赛旗】尊敬的用户，您好！您购买的艺赛旗机器人流程挖掘平台已为您开通服务，服务有效期截止至：YYYYMMDD。您可以使用艺赛旗帐号(帐号ID: xxxxxx) 登录https://cloud.i-search.com.cn，祝您使用愉快！回TD退订。

         - 邮件内容如下：





3、用户登录

当用户输入艺赛旗ID及密码通过身份认证后，工作台调取RPM服务接口获取当前艺赛旗ID的RPM租户信息以及用户信息，并进入选择租户界面，如下图：




（1）新邀请

若当前艺赛旗ID有未处理的新RPM租户邀请，则显示新的邀请信息：租户名称、租户类型（RPM）、租户访问地址

（注：需要将原RPM租户的访问地址，例如：“https://rpm.i-search.com.cn/租户编号”，转成cloud域名地址）

- 点击【接受】，“新邀请”列表中的邀请信息消失，将当前租户加入下方允许访问的租户列表

- 点击【拒绝】，“新邀请”列表中的邀请信息消失

（2）允许访问的租户列表
该列表中增加显示当前艺赛旗ID可访问的租户列表（一个艺赛旗ID可加入N个RPM租户），每个租户分别显示：租户名称、租户类型（RPM）、租户访问地址

- 点击【单行租户】则直接跳转进入当前租户的RPM服务平台

- 当租户许可过期时，租户名称旁边显示“已过期”标签

- 点击已过期的租户跳转至RPM平台后显示无访问权限提示页，倒数5秒后自动跳转回工作台，且支持手动点击按钮【立即跳转】，如下图：

（注：当用户获取到RPM服务的长链接地址直接访问时，也需要判断当前用户是否有访问权限，无权限则显示同样的舞访问权限提示页）





4、租户管理

当用户在登录页面选择快捷入口【登录至工作台】时，则页面跳转至“工作台总览”，总览页面显示内容分为以下两种：



租户管理员
租户管理原账号登录工作台后，总览页面显示当前账号的RPM租户信息，如下图：


（1）租户信息内容：租户名称、租户类型（RPM）、当前租户的许可类型（正式、试用）、租户号（支持复制）、域名、租户到期日

         - 租户号支持一键复制

         - 域名支持点击跳转至RPM服务，且支持复制

         - 租户到期日在到期前7天开始显示为红色

（2）租户到期日前7天页面顶部显示横幅提示，内容如下：您的租户“xxxxxxx”将在7天后过期，请进行续费，如需帮助，请联系您的客户经理或拨打客服热线 400-821-8738 咨询

（3）当前租户的许可过期后，该租户左上角角标提示当前租户：已过期

（4）许可详情：点击【许可详情】实时获取当前租户的许可消耗情况，如下图：

         - 许可内容包含：发现机器人、分析用户数、分析流程数、日志数据存储

         - 分别显示没中类型的许可数量总数，以及当前已使用的数量



（5）每个租户支持的操作如下：

        - 未过期 租户支持的操作有：用户管理、购买、操作日志、租户转移

                      用户管理：点击【用户管理】跳转至用户管理列表并选中当前的RPM租户，支持【邀请用户】

                      购买：点击【购买】跳转至购买页面，支持扫码添加企业微信，支持点击【回到工作台】跳转为总览

                      操作日志：记录当前租户号的所有操作数据，包括用户管理及租户转移记录

                      租户转移：支持填写目标账号、获取目标账号验证码、当前账号密码（未设置密码支持跳转至“安全设置”）

                      确认提交后，租户移交成功（注：租户移交成功后，当前账号将无法再总览中查看该租户信息且与当前租户解绑）

                      注：点击获取验证码按钮时，系统需要校验目标账号是否为当前租户的分析师角色

                      - 若目标账号为当前租户的分析师则报错提示：目标账号为当前租户的分析师，请修改其角色后重试

                      - 若目标账号未加入当前租户或为当前租户管理员，则发送验证码短信或邮件给目标账号


                        短信内容：【艺赛旗】您的验证码是168907（15分钟内有效）。目前正在使用租户转移功能，请谨慎将验证码提供给他人。

                        邮件内容：

                        




        - 已过期 租户支持的操作有：用户管理、购买、操作日志、租户转移、删除

           用户管理：进入用户管理页面后，已过期的租户无法再邀请新用户

        （注：租户过期后，已发送且7天内未处理的用户邀请链接全部失效）

           购买：点击【购买】跳转至购买页面，支持扫码添加企业微信，支持点击【回到工作台】跳转为总览

           操作日志：已过期的租户仍可查看操作日志

           租户转移：已过期的租户同样支持租户转移功能

           删除：需要用户填写艺赛旗ID及密码进行二次确认





普通用户
普通用户账号登录工作台后，总览页面的租户信息栏则不显示其加入的租户信息，并提示：您当前无开通服务，请拨打 400-8218-738 或者在线【申请试用】


In [7]:
requirements = '''功能简介：



5、用户管理

用户管理页面列表同样需要根据当前登录的账号信息判断显示内容，具体内容如下：



普通用户账号登录时，用户管理列表显示“暂无数据”
租户管理员账号登录时，用户管理列表现在当前租户下的所有用户数据，并支持对用户列表进行增删改查操作，如下图：

用户列表

（1）用户列表通过tab标签页区分RPM和RPA

（2）租户切换：RPM的tab标签页中仅支持切换RPM租户并显示对应的RPM用户列表，RPA的tab标签页中仅显示RPA的租户并显示对应的RPA用户列表

（3）搜索：支持通过“姓名”或“艺赛旗ID”关键词对每个租户的用户列表进行模糊搜索

（4）用户列表字段：艺赛旗ID、姓名、角色、用户状态、上次登录、操作

         - 艺赛旗ID：管理员用户ID需要以标签形式突出账号身份，标签内容：“租户管理员”；其他普通用户显示艺赛旗ID账号（注：租户管理员账号置顶显示）

         - 姓名：用户别名，由用户个人在账户中心中修改，未填写时显示为“-”

         - 角色：RPM管理员、RPM分析师，租户管理员默认角色为RPM管理员

         - 用户状态：正常、邀请中、失败、失效

         - 上次登录：状态为“正常”的用户上次登录显示最后一次登录工作台的时间；

                             状态为“邀请中”的用户显示“待确认”，并显示按钮【重新发送】，点击后账号为手机号的用户发送邀请短信，账号为邮箱的用户发送邀请邮件

                             状态为“失效、失败”的用户显示按钮【重新发送】，点击后账号为手机号的用户发送邀请短信，账号为邮箱的用户发送邀请邮件

         - 操作：租户管理员账号无操作按钮

                      状态为“正常”的用户支持点击【编辑】按钮修改角色（其他信息不可修改），支持点击【删除】按钮在用户列表中删除当前用户，并解绑该艺赛旗ID在当前租户中的许可

                      状态为“邀请中”的用户支持点击【撤销】按钮撤销当前邀请（撤销后邀请链接失效，且用户状态变为“失效”），支持点击【删除】按钮在用户列表中删除当前用户

                      状态为“失效、失败”的用户支持点击【删除】按钮在用户列表中删除当前用户，并解绑该艺赛旗ID在当前租户中的许可

（5）邀请用户：点击【邀请用户】按钮，弹出侧边栏内容如下图：



- 邀请用户表单内容如下：租户名称（置灰不可修改）、艺赛旗ID（必填，手机号或邮箱）、角色（RPM分析师、RPM管理员）

- 每次邀请时，显示当前租户的许可剩余数量（分析用户数剩余数量）

- 确认提交后系统tips提示：用户邀请已发送，同时发送对应的邮件或短信给相应的艺赛旗ID（若用户未处理短信或邮件则在登录时提示“新邀请”信息）

（已过期的租户，提交表单时报错提示：当前租户许可已过期，无法邀请新用户）






6、我的租户

我的租户列表中加入当前登录账号拥有的RPM租户以及加入过的租户，如下图：



（1）搜索：支持通过“租户名称”字段关键词模糊搜索

（2）列表字段包含：租户名、访问地址、租户到期日

         - 租户名：需要以标签区分出RPA及RPM租户

         - 访问地址：支持点击链接跳转至RPM服务，且支持一键复制地址

         - 租户到期日：显示当前租户的到期时间





7、RPM服务平台修改

（1）RPM平台中同步工作台中的用户个人信息，需求详情见：http://192.168.0.201:8088/zentao/story-view-6706.html

（2）RPM平台的“访问用户“管理菜单中列表UI样式、字段及操作修改（私有化及公有云都改），需求详情见：http://192.168.0.201:8088/zentao/story-view-6880.html
'''

result = overall_chain({'requirements': requirements})



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='test0406.openai.azure.com', port=443): Read timed out. (read timeout=600).



> Finished chain.


In [8]:
print(result['feature'])

根据需求说明书，我的选择是"流程挖掘和任务挖掘"这一核心功能点。这个功能点可以从业务日志和UI操作日志中还原出业务流程并进行分析。在业务场景中，常常需要了解业务的流程和任务，以便优化流程，提高效率。而通过这个功能点，可以快速还原出业务流程，减轻了用户自行分析的负担，提高了工作效率，具备创新潜力。此外，在新的业务场景下，也可以选择新的算法，实现更高效、准确的业务流程还原和分析。


In [9]:
print(result['solution'])

产品经理1的思路描述：

问题描述：在企业日常运营中，业务流程复杂，任务繁多，需要对流程和任务进行分析和优化，以提高效率和降低成本。然而，传统的业务流程分析和优化方法较为繁琐，需要人力投入大量时间和精力，效率低下。

解决思路：利用机器学习和数据挖掘技术，开发一套流程挖掘和任务挖掘的自动化分析工具。通过对业务日志和UI操作日志的分析，可以自动还原出业务流程，并对任务进行分类和优化。同时，该工具可以根据用户需求进行灵活定制，提高分析和优化的准确性和效率。

创新性体现：该解决思路采用了机器学习和数据挖掘技术，不仅提高了分析和优化的效率，也提高了准确性。此外，该工具具有灵活性，可以根据不同用户需求进行定制，更好地满足企业实际需求。

产品经理2的思路描述：

问题描述：在企业日常运营中，业务流程繁琐，任务众多，人工分析和优化效率低下，难以满足企业实际需求。

解决思路：基于图神经网络技术，开发一套流程挖掘和任务挖掘的自动化分析工具。该工具可以自动还原出业务流程，并对任务进行分析和优化。同时，该工具可以根据用户需求进行灵活定制，提高分析和优化的准确性和效率。

创新性体现：该解决思路采用了图神经网络技术，可以更好地处理业务流程中的复杂关系，提高分析和优化的效率和准确性。同时，该工具具有灵活性，可以根据不同用户需求进行定制，更好地满足企业实际需求。

产品经理3的思路描述：

问题描述：在企业日常运营中，业务流程繁琐，任务众多，人工分析和优化效率低下，难以满足企业实际需求。

解决思路：利用深度学习和自然语言处理技术，开发一套流程挖掘和任务挖掘的自动化分析工具。该工具可以自动还原出业务流程，并对任务进行分类和优化，同时可以通过自然语言处理技术进行智能推荐和分析。该工具可以根据用户需求进行灵活定制，提高分析和优化的准确性和效率。

创新性体现：该解决思路采用了深度学习和自然语言处理技术，可以更好地处理业务流程中的语言信息，提高分析和优化的效率和准确性。同时，该工具具有智能推荐功能，可以根据用户需求推荐最优解决方案，更好地满足企业实际需求。


In [10]:
print(result['selection'])

问题描述：在企业日常运营中，业务流程复杂，任务繁多，需要对流程和任务进行分析和优化，以提高效率和降低成本。然而，传统的业务流程分析和优化方法较为繁琐，需要人力投入大量时间和精力，效率低下。

解决方案：我们将采用基于深度学习和自然语言处理技术的流程挖掘和任务挖掘的自动化分析工具。该工具可以自动还原出业务流程，并对任务进行分类和优化，同时可以通过自然语言处理技术进行智能推荐和分析。该工具可以根据用户需求进行灵活定制，提高分析和优化的准确性和效率。

价值体现：该解决方案通过深度学习和自然语言处理技术，可以更好地处理业务流程中的语言信息，提高分析和优化的效率和准确性。同时，该工具具有智能推荐功能，可以根据用户需求推荐最优解决方案，并根据不同用户需求进行定制，更好地满足企业实际需求，提高企业运营效率和降低成本。


In [11]:
with open('patents/requirements-test4.txt', 'w') as f:
    f.write('requirements: ' + result['requirements'])
    f.write('\n\nfeature: ' + result['feature'])
    f.write('\n\nsolution: ' + result['solution'])
    f.write('\n\nselection: ' + result['selection'])